<h1>Chapter 06. Object references, modifiability and reuse.</h1>

<h2>Variables are not boxes</h2>

Variables `a` and `b` store references to the same list, not copies of the list

In [1]:
a = [1, 2, 3]
b = a

b.append(4)
b

[1, 2, 3, 4]

Variables are bounded to objects only after objects are created

In [2]:
class Gizmo:
    def __init__(self):
        print(f"Gizmo id: {id(self)}")

In [3]:
x = Gizmo()

Gizmo id: 4538703296


In [4]:
try:
    y = Gizmo() * 10
except TypeError as e:
    print(e.__repr__())

Gizmo id: 4538702432
TypeError("unsupported operand type(s) for *: 'Gizmo' and 'int'")


The second `Gizmo` object was still created before the attempt to perform multiplication. But the `y` varaible was never created because an exeception occured when the right part was being calculated.

<h2>Identity, Equality and Pseudonyms</h2>

In [5]:
charles = {
    'name': 'Charles L. Dodgson',
    'born': 1832
}

In [6]:
lewis = charles

lewis is charles

True

In [7]:
id(charles), id(lewis)

(4538736128, 4538736128)

In [8]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [9]:
alex = {
    'name': 'Charles L. Dodgson',
    'born': 1832,
    'balance': 950
}

alex == charles

True

In [10]:
alex is charles

False

<h3>Choice between <code>==</code> and <code>is</code></h3>

The `==` operator compares the values of objects (the data stored in them), and the `is` operator compares their identifiers.

<h3>Relative immutability of tuples</h3>

In [11]:
t1 = (1, 2, [3, 4])
t2 = (1, 2, [3, 4])

t1 == t2

True

In [12]:
id(t1[-1])

4538171136

t1[-1].append(5)
t1

In [13]:
id(t1[-1])

4538171136

In [14]:
t1 == t2

True

The object identifier `t1[-1]` has not changed only its value has changed.
`t1` and `t2` are now not equal.

<h2>By default, copying is superficial</h2>

In [15]:
l1 = [3, [55, 44], (7, 8, 9)]

In [16]:
l2 = list(l1)  # l2 = l1[:]

In [17]:
l2

[3, [55, 44], (7, 8, 9)]

In [18]:
l2 == l1

True

In [19]:
l2 is l1

False

<h3>Deep and superficial copying of objects</h3>

In [20]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

`copy.copy()` creates a shallow copy of the object, duplicating the object itself but not recursively copying inner objects.
`copy.deepcopy()` creates a deep copy of the object, recursively copying all inner objects as well, ensuring changes to the copied object won't affect the original.

In [21]:
import copy


bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

In [22]:
id(bus1), id(bus2), id(bus3)

(4538710160, 4538897840, 4538896640)

In [23]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(4538870464, 4538870464, 4538536000)

In [24]:
bus1.drop('Bill')

In [25]:
bus2.passengers

['Alice', 'Claire', 'David']

In [26]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [27]:
bus2.pick('Julie')

In [28]:
bus1.passengers

['Alice', 'Claire', 'David', 'Julie']

In [29]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

Cyclic references: `b` refers to `a` and then appends to the end of `a`; however `deepcopy` handles the copyinf of `a`

In [30]:
a = [10, 20]
b = [a, 30]

In [31]:
a.append(b)
a

[10, 20, [[...], 30]]

In [32]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]